In [1]:
from nba_api.stats.static import teams

nba_teams = teams.get_teams()

denver  = [team for team in nba_teams if team ['abbreviation'] == 'DEN'][0]
denver_id = denver['id']

In [2]:
from nba_api.stats.endpoints import leaguegamefinder

gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=denver_id)
games = gamefinder.get_data_frames()[0]
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22025,1610612743,DEN,Denver Nuggets,0022500752,2026-02-07,DEN @ CHI,W,240,136,...,0.667,11,30,41,37,7,7,13,17,16.0
1,22025,1610612743,DEN,Denver Nuggets,0022500726,2026-02-04,DEN @ NYK,L,291,127,...,0.815,15,36,51,29,8,3,14,21,-7.0
2,22025,1610612743,DEN,Denver Nuggets,0022500716,2026-02-03,DEN @ DET,L,240,121,...,0.860,12,31,43,20,7,10,10,20,-3.0
3,22025,1610612743,DEN,Denver Nuggets,0022500711,2026-02-01,DEN vs. OKC,L,238,111,...,0.708,8,31,39,29,9,4,17,20,-10.0
4,22025,1610612743,DEN,Denver Nuggets,0022500695,2026-01-30,DEN vs. LAC,W,240,122,...,0.813,7,36,43,30,7,4,12,18,13.0


In [3]:
# Query for the play by play of that most recent regular season game
from nba_api.stats.endpoints import playbyplayv3
game_id = '0022500711'
df = playbyplayv3.PlayByPlayV3(game_id).get_data_frames()[0]
df[[
    'personId',
    'playerName']
]

,personId,playerName
0,0,
1,203999,Jokić
2,1628392,Hartenstein
3,1631212,Watson
4,1631212,Watson
...,...,...
484,1642964,Barnhizer
485,1631124,Strawther
486,1610612743,
487,1631124,Strawther


In [4]:

p_name = 'Watson'
df[df['playerName'].str.contains(p_name,na=False)]

player_df = df[df['playerName'].str.contains(p_name, na=False)]




In [5]:
player_df = player_df.sort_values(
    by = ['period', 'actionNumber']
)

In [6]:
app_action_types = [
    'Made Shot',
    'Missed Shot',
    'Turnover',
    'Free Throw',
    'Rebound',
    'Foul'
]

In [7]:
keep_mask = (
    player_df['actionType'].isin(app_action_types)
    |
    player_df['description'].str.contains('STEAL|BLOCK', na=False)
)

In [8]:
filtered_df = player_df[keep_mask]
filtered_df = filtered_df.reset_index(drop=True)
filtered_df['s_no'] = filtered_df.index + 1
filtered_df[filtered_df['actionType'] == 'Substitution']

filtered_df[
    [
        'personId',
        'period',
        'clock',
        'actionType',
        'subType',
        'shotResult',
        'shotDistance',
        'shotValue',
        'description'
    ]
]



,personId,period,clock,actionType,subType,shotResult,shotDistance,shotValue,description
0,1631212,1,PT11M46.00S,,,,0,0,Watson STEAL (1 STL)
1,1631212,1,PT11M38.00S,Missed Shot,Cutting Layup Shot,Missed,3,2,MISS Watson 3' Cutting Layup Shot
2,1631212,1,PT10M37.00S,Missed Shot,Running Pull-Up Jump Shot,Missed,26,3,MISS Watson 26' 3PT Running Pull-Up Jump Shot
3,1631212,1,PT08M52.00S,Made Shot,Jump Shot,Made,25,3,Watson 25' 3PT Jump Shot (3 PTS) (Pickett 1 AST)
4,1631212,1,PT07M39.00S,Made Shot,Cutting Dunk Shot,Made,0,2,Watson Cutting Dunk Shot (5 PTS) (Jokic 2 AST)
5,1631212,1,PT06M08.00S,Missed Shot,Step Back Jump shot,Missed,23,2,MISS Watson 23' Step Back Jump Shot
6,1631212,1,PT04M47.00S,Rebound,Unknown,,0,0,Watson REBOUND (Off:0 Def:1)
7,1631212,1,PT03M46.00S,Made Shot,Pullup Jump shot,Made,8,2,Watson 8' Pullup Jump Shot (7 PTS) (Murray 2 AST)
8,1631212,1,PT03M07.00S,Made Shot,Jump Shot,Made,24,3,Watson 24' 3PT Jump Shot (10 PTS) (Valanciunas...
9,1631212,1,PT01M41.00S,Foul,Shooting,,0,0,Watson S.FOUL (P1.PN) (B.Schwab)


In [9]:
player_event_dict = {}

for person_id, group in filtered_df.groupby('personId'):
    player_event_dict[person_id] = group.reset_index(drop=True)

In [10]:
import pickle 
with open ('player_event_dict.pkl', 'wb') as f:
    pickle.dump(player_event_dict, f)

In [11]:
import pickle
##WATSON
app_data = {
    "0022500711": {   # gameId
        1631212: {    # personId (Watson)
            "final_rating": 4.27,
            "events": [
                {"period": 1, "clock": "10:51", "ui_event": "24' corner three", "delta": 0.30},
                {"period": 1, "clock": "8:54",  "ui_event": "Missed 15' drive floater", "delta": 0.07},
                {"period": 1, "clock": "6:18",  "ui_event": "25' fastbreak lead-taking three", "delta": 0.23},
                {"period": 1, "clock": "5:59",  "ui_event": "Missed 24' lead-taking three", "delta": 0.07},
                {"period": 1, "clock": "4:43",  "ui_event": "Turnover - Lost ball", "delta": -0.18},
                {"period": 1, "clock": "3:38",  "ui_event": "2/2 free throws", "delta": 0.18},
                {"period": 1, "clock": "2:31",  "ui_event": "6' drive floater bank", "delta": 0.19},

                {"period": 2, "clock": "3:29",  "ui_event": "6' fastbreak two", "delta": 0.12},
                {"period": 2, "clock": "3:04",  "ui_event": "Turnover", "delta": -0.18},
                {"period": 2, "clock": "1:42",  "ui_event": "24' three", "delta": 0.24},
                {"period": 2, "clock": "31.1",  "ui_event": "Missed 30' bomb", "delta": 0.07},
                {"period": 2, "clock": "26.3",  "ui_event": "Fastbreak layup", "delta": 0.13},

                {"period": 3, "clock": "11:30", "ui_event": "17' two", "delta": 0.17},
                {"period": 3, "clock": "11:00", "ui_event": "Drive lead-taking layup and-1", "delta": 0.22},
                {"period": 3, "clock": "8:11",  "ui_event": "Missed 27' lead-taking three", "delta": 0.07},
                {"period": 3, "clock": "6:56",  "ui_event": "26' three", "delta": 0.27},
                {"period": 3, "clock": "6:27",  "ui_event": "Steal", "delta": 0.19},
                {"period": 3, "clock": "2:52",  "ui_event": "Missed 12' drive game-tying floater", "delta": 0.15},
                {"period": 3, "clock": "2:37",  "ui_event": "Fastbreak game-tying dunk", "delta": 0.16},
                {"period": 3, "clock": "1:57",  "ui_event": "Missed 24' lead-taking three", "delta": 0.07},
                {"period": 3, "clock": "1:51",  "ui_event": "Alley-oop fastbreak layup", "delta": 0.13},
                {"period": 3, "clock": "1:10",  "ui_event": "24' corner three", "delta": 0.26},
                {"period": 3, "clock": "40.3",  "ui_event": "27' three", "delta": 0.25},
                {"period": 3, "clock": "6.2",   "ui_event": "24' corner three", "delta": 0.29},

                {"period": 4, "clock": "5:57",  "ui_event": "9' fadeaway", "delta": 0.23},
                {"period": 4, "clock": "5:11",  "ui_event": "26' three", "delta": 0.25},
                {"period": 4, "clock": "4:29",  "ui_event": "11' two", "delta": 0.11}
            ]
        }
    }
}




In [12]:
#HARTINSTIEN
app_data["0022500711"][1628392] = {
    "final_rating": 0.91,
    "events": [
        {"period": 1, "clock": "11:46", "ui_event": "Turnover - Bad pass", "delta": -0.18},
        {"period": 1, "clock": "10:37", "ui_event": "Missed 26' fastbreak lead-taking three", "delta": 0.07},
        {"period": 1, "clock": "10:14", "ui_event": "Missed 15' fadeaway", "delta": 0.15},
        {"period": 1, "clock": "10:12", "ui_event": "Putback layup", "delta": 0.20},

        {"period": 2, "clock": "4:44",  "ui_event": "Turnover - Bad pass", "delta": -0.19},
        {"period": 2, "clock": "1:20",  "ui_event": "Missed drive reverse layup", "delta": 0.07},

        {"period": 3, "clock": "11:01", "ui_event": "26' three", "delta": 0.34},
        {"period": 3, "clock": "9:20",  "ui_event": "Layup", "delta": 0.20},
        {"period": 3, "clock": "6:48",  "ui_event": "Turnover - Offensive foul", "delta": -0.19},

        {"period": 4, "clock": "7:37",  "ui_event": "Missed 8' drive layup", "delta": 0.07},
        {"period": 4, "clock": "7:24",  "ui_event": "Missed 27' three", "delta": 0.15},
        {"period": 4, "clock": "4:09",  "ui_event": "Steal", "delta": 0.20}
    ]
}



In [13]:
#JOKIC

app_data["0022500711"][203999] = {
    "final_rating": 2.99,
    "events": [
        {"period": 1, "clock": "9:59", "ui_event": "12' two", "delta": 0.13},
        {"period": 1, "clock": "7:39", "ui_event": "Dunk", "delta": 0.14},
        {"period": 1, "clock": "6:21", "ui_event": "Missed 24' corner three", "delta": 0.15},
        {"period": 1, "clock": "5:44", "ui_event": "Turnover - Bad pass", "delta": -0.17},
        {"period": 1, "clock": "5:10", "ui_event": "8' hook", "delta": 0.19},
        {"period": 1, "clock": "4:09", "ui_event": "25' corner three", "delta": 0.18},

        {"period": 2, "clock": "6:31", "ui_event": "27' three", "delta": 0.18},
        {"period": 2, "clock": "5:38", "ui_event": "24' corner three", "delta": 0.28},
        {"period": 2, "clock": "5:04", "ui_event": "26' three", "delta": 0.17},
        {"period": 2, "clock": "4:03", "ui_event": "Turnover - Out-of-bounds bad pass", "delta": -0.17},
        {"period": 2, "clock": "3:37", "ui_event": "Turnover - Offensive foul", "delta": -0.17},
        {"period": 2, "clock": "2:58", "ui_event": "Steal", "delta": 0.17},
        {"period": 2, "clock": "2:01", "ui_event": "2/2 free throws", "delta": 0.18},
        {"period": 2, "clock": "38.4", "ui_event": "Turnover - Offensive foul", "delta": -0.18},
        {"period": 2, "clock": "21.7", "ui_event": "6' floater", "delta": 0.25},

        {"period": 3, "clock": "11:48", "ui_event": "19' two", "delta": 0.32},
        {"period": 3, "clock": "9:02", "ui_event": "1/2 free throws", "delta": 0.09},
        {"period": 3, "clock": "8:33", "ui_event": "27' three", "delta": 0.21},
        {"period": 3, "clock": "7:20", "ui_event": "Putback layup", "delta": 0.22},
        {"period": 3, "clock": "7:05", "ui_event": "Missed drive layup", "delta": 0.07},
        {"period": 3, "clock": "6:02", "ui_event": "26' three", "delta": 0.41},

        {"period": 4, "clock": "9:53", "ui_event": "Missed 24' three", "delta": 0.07},
        {"period": 4, "clock": "9:36", "ui_event": "10' drive floater", "delta": 0.21},
        {"period": 4, "clock": "5:32", "ui_event": "Missed fastbreak layup", "delta": 0.07},
        {"period": 4, "clock": "5:26", "ui_event": "Fastbreak layup", "delta": 0.17},
        {"period": 4, "clock": "4:09", "ui_event": "Turnover - Bad pass", "delta": -0.17},
        {"period": 4, "clock": "3:55", "ui_event": "Missed 29' bomb", "delta": 0.07},
        {"period": 4, "clock": "3:23", "ui_event": "Missed 24' corner three", "delta": 0.07},
        {"period": 4, "clock": "3:15", "ui_event": "Turnover - Lost ball", "delta": -0.17}
    ]
}


In [14]:
app_data["0022500711"][203501] = {
    "final_rating": 1.07,
    "events": [
        {"period": 1, "clock": "5:30",  "ui_event": "Missed 26' three", "delta": 0.07},
        {"period": 1, "clock": "45.2",  "ui_event": "2/2 free throws", "delta": 0.17},
        {"period": 1, "clock": "30.6",  "ui_event": "Blocked dunk", "delta": 0.17},
        {"period": 1, "clock": "2.6",   "ui_event": "Steal", "delta": 0.17},

        {"period": 2, "clock": "11:27", "ui_event": "Steal", "delta": 0.18},
        {"period": 2, "clock": "11:23", "ui_event": "Turnover - Bad pass", "delta": -0.18},
        {"period": 2, "clock": "3:23",  "ui_event": "Steal", "delta": 0.17},
        {"period": 2, "clock": "3:21",  "ui_event": "1/2 fastbreak free throws", "delta": 0.08},
        {"period": 2, "clock": "1:36",  "ui_event": "Missed 28' bomb", "delta": 0.07},
        {"period": 2, "clock": "51.2",  "ui_event": "Missed 27' three", "delta": 0.07},

        {"period": 4, "clock": "7:14",  "ui_event": "Missed 30' bomb", "delta": 0.07}
    ]
}


In [15]:
import pandas as pd

rows = []

for game_id, players in app_data.items():
    for person_id, pdata in players.items():
        for ev in pdata["events"]:
            ui = ev["ui_event"].lower()

            # exclude non-shot events
            if any(x in ui for x in ["missed", "turnover", "steal", "free throw", "block"]):
                continue

            rows.append({
                "gameId": game_id,
                "personId": person_id,
                "period": ev["period"],
                "clock": ev["clock"],
                "ui_event": ev["ui_event"],
                "delta": ev["delta"]
            })

shots_made_df = pd.DataFrame(rows)

PLAYER_MAP = {
    1631212: "Watson",
    203999: "Jokic",
    1628392: "Hartenstein",
    203501: "Hardaway Jr."
}
shots_made_df["player"] = shots_made_df["personId"].map(PLAYER_MAP)
shots_made_df = shots_made_df[
    ["player", "period", "clock", "ui_event", "delta"]
]


shots_made_df


,player,period,clock,ui_event,delta
0,Watson,1,10:51,24' corner three,0.30
1,Watson,1,6:18,25' fastbreak lead-taking three,0.23
2,Watson,1,2:31,6' drive floater bank,0.19
3,Watson,2,3:29,6' fastbreak two,0.12
4,Watson,2,1:42,24' three,0.24
5,Watson,2,26.3,Fastbreak layup,0.13
6,Watson,3,11:30,17' two,0.17
7,Watson,3,11:00,Drive lead-taking layup and-1,0.22
8,Watson,3,6:56,26' three,0.27
9,Watson,3,2:37,Fastbreak game-tying dunk,0.16


In [16]:
import pandas as pd

rows = []

for game_id, players in app_data.items():
    for person_id, pdata in players.items():
        for ev in pdata["events"]:
            ui = ev["ui_event"].lower()

            # keep only missed shots (exclude free throws)
            if "missed" not in ui:
                continue
            if "free throw" in ui:
                continue

            rows.append({
                "gameId": game_id,
                "personId": person_id,
                "period": ev["period"],
                "clock": ev["clock"],
                "ui_event": ev["ui_event"],
                "delta": ev["delta"]
            })

shots_missed_df = pd.DataFrame(rows)

shots_missed_df["player"] = shots_missed_df["personId"].map(PLAYER_MAP)
shots_missed_df = shots_missed_df[
    ["player", "period", "clock", "ui_event", "delta"]
]

shots_missed_df


,player,period,clock,ui_event,delta
0,Watson,1,8:54,Missed 15' drive floater,0.07
1,Watson,1,5:59,Missed 24' lead-taking three,0.07
2,Watson,2,31.1,Missed 30' bomb,0.07
3,Watson,3,8:11,Missed 27' lead-taking three,0.07
4,Watson,3,2:52,Missed 12' drive game-tying floater,0.15
5,Watson,3,1:57,Missed 24' lead-taking three,0.07
6,Hartenstein,1,10:37,Missed 26' fastbreak lead-taking three,0.07
7,Hartenstein,1,10:14,Missed 15' fadeaway,0.15
8,Hartenstein,2,1:20,Missed drive reverse layup,0.07
9,Hartenstein,4,7:37,Missed 8' drive layup,0.07


In [17]:
import pandas as pd

rows = []

for game_id, players in app_data.items():
    for person_id, pdata in players.items():
        for ev in pdata["events"]:
            ui = ev["ui_event"].lower()

            # keep only defensive events
            if not any(x in ui for x in ["steal", "block"]):
                continue

            rows.append({
                "gameId": game_id,
                "personId": person_id,
                "period": ev["period"],
                "clock": ev["clock"],
                "ui_event": ev["ui_event"],
                "delta": ev["delta"]
            })

defensive_df = pd.DataFrame(rows)

defensive_df["player"] = defensive_df["personId"].map(PLAYER_MAP)
defensive_df = defensive_df[
    ["player", "period", "clock", "ui_event", "delta"]
]

defensive_df


,player,period,clock,ui_event,delta
0,Watson,3,6:27,Steal,0.19
1,Hartenstein,4,4:09,Steal,0.20
2,Jokic,2,2:58,Steal,0.17
3,Hardaway Jr.,1,30.6,Blocked dunk,0.17
4,Hardaway Jr.,1,2.6,Steal,0.17
5,Hardaway Jr.,2,11:27,Steal,0.18
6,Hardaway Jr.,2,3:23,Steal,0.17


In [18]:
import pandas as pd

rows = []

for game_id, players in app_data.items():
    for person_id, pdata in players.items():
        for ev in pdata["events"]:
            ui = ev["ui_event"].lower()

            # keep only free throw events
            if "free throw" not in ui:
                continue

            rows.append({
                "gameId": game_id,
                "personId": person_id,
                "period": ev["period"],
                "clock": ev["clock"],
                "ui_event": ev["ui_event"],
                "delta": ev["delta"]
            })

free_throws_df = pd.DataFrame(rows)

free_throws_df["player"] = free_throws_df["personId"].map(PLAYER_MAP)
free_throws_df = free_throws_df[
    ["player", "period", "clock", "ui_event", "delta"]
]

free_throws_df


,player,period,clock,ui_event,delta
0,Watson,1,3:38,2/2 free throws,0.18
1,Jokic,2,2:01,2/2 free throws,0.18
2,Jokic,3,9:02,1/2 free throws,0.09
3,Hardaway Jr.,1,45.2,2/2 free throws,0.17
4,Hardaway Jr.,2,3:21,1/2 fastbreak free throws,0.08


In [19]:
import pandas as pd

rows = []

for game_id, players in app_data.items():
    for person_id, pdata in players.items():
        for ev in pdata["events"]:
            ui = ev["ui_event"].lower()

            # keep only turnovers
            if "turnover" not in ui:
                continue

            rows.append({
                "gameId": game_id,
                "personId": person_id,
                "period": ev["period"],
                "clock": ev["clock"],
                "ui_event": ev["ui_event"],
                "delta": ev["delta"]
            })

turnovers_df = pd.DataFrame(rows)

PLAYER_MAP = {
    1631212: "Watson",
    203999: "Jokic",
    1628392: "Hartenstein",
    203501: "Hardaway Jr."
}

turnovers_df["player"] = turnovers_df["personId"].map(PLAYER_MAP)
turnovers_df = turnovers_df[
    ["player", "period", "clock", "ui_event", "delta"]
]

turnovers_df


,player,period,clock,ui_event,delta
0,Watson,1,4:43,Turnover - Lost ball,-0.18
1,Watson,2,3:04,Turnover,-0.18
2,Hartenstein,1,11:46,Turnover - Bad pass,-0.18
3,Hartenstein,2,4:44,Turnover - Bad pass,-0.19
4,Hartenstein,3,6:48,Turnover - Offensive foul,-0.19
5,Jokic,1,5:44,Turnover - Bad pass,-0.17
6,Jokic,2,4:03,Turnover - Out-of-bounds bad pass,-0.17
7,Jokic,2,3:37,Turnover - Offensive foul,-0.17
8,Jokic,2,38.4,Turnover - Offensive foul,-0.18
9,Jokic,4,4:09,Turnover - Bad pass,-0.17


In [20]:
api_shots_df = df[
    df["actionType"].isin(["Made Shot", "Missed Shot"])
].copy()


In [21]:
api_shots_df = api_shots_df[
    [
        "personId",
        "period",
        "clock",
        "actionType",
        "subType",
        "shotDistance",
        "shotValue",
        "description"
    ]
]
api_shots_df.head()



,personId,period,clock,actionType,subType,shotDistance,shotValue,description
4,1631212,1,PT11M38.00S,Missed Shot,Cutting Layup Shot,3,2,MISS Watson 3' Cutting Layup Shot
7,1641717,1,PT11M22.00S,Made Shot,Jump Shot,27,3,Wallace 27' 3PT Jump Shot (3 PTS) (Gilgeous-Al...
8,1629618,1,PT11M01.00S,Made Shot,Cutting Layup Shot,3,2,Pickett 3' Cutting Layup Shot (2 PTS) (Murray ...
9,1631096,1,PT10M44.00S,Missed Shot,Jump Shot,27,3,MISS Holmgren 27' 3PT Jump Shot
11,1631212,1,PT10M37.00S,Missed Shot,Running Pull-Up Jump Shot,26,3,MISS Watson 26' 3PT Running Pull-Up Jump Shot


In [22]:
api_shots_df["actionType"].value_counts()


actionType
Missed Shot    85
Made Shot      79
Name: count, dtype: int64

In [23]:
made_shots_api = api_shots_df[
    api_shots_df["actionType"] == "Made Shot"
].copy()

missed_shots_api = api_shots_df[
    api_shots_df["actionType"] == "Missed Shot"
].copy()


In [24]:
made_shots_api["shot_family"] = made_shots_api["shotValue"].map({
    2: "2PT",
    3: "3PT"
})

missed_shots_api["shot_family"] = missed_shots_api["shotValue"].map({
    2: "2PT",
    3: "3PT"
})


In [25]:
def normalize_subtype(st):
    if pd.isna(st):
        return "unknown"

    st = st.lower()

    if "dunk" in st:
        return "dunk"
    if "layup" in st:
        return "layup"
    if "floater" in st or "floating" in st:
        return "floater"
    if "hook" in st:
        return "hook"
    if "pull" in st:
        return "pullup"
    if "jump" in st:
        return "jump_shot"

    return "other"


In [26]:
made_shots_api["shot_subtype"] = made_shots_api["subType"].apply(normalize_subtype)
missed_shots_api["shot_subtype"] = missed_shots_api["subType"].apply(normalize_subtype)


In [27]:
made_shots_api.groupby(["shot_family", "shot_subtype"]).size()


shot_family  shot_subtype
2PT          dunk             3
             floater          7
             hook             2
             jump_shot       10
             layup           15
             other            1
             pullup           6
3PT          jump_shot       34
             pullup           1
dtype: int64

In [28]:
missed_shots_api.groupby(["shot_family", "shot_subtype"]).size()


shot_family  shot_subtype
2PT          dunk             1
             floater          1
             hook             2
             jump_shot        4
             layup           18
             other            3
             pullup           8
3PT          jump_shot       38
             pullup          10
dtype: int64

In [29]:
rated_players = set([
    1631212,  # Watson
    203999,   # Jokic
    1628392,  # Hartenstein
    203501    # Hardaway Jr.
])


In [30]:
api_shots_df = api_shots_df[
    api_shots_df["personId"].isin(rated_players)
]

made_shots_api = made_shots_api[
    made_shots_api["personId"].isin(rated_players)
]

missed_shots_api = missed_shots_api[
    missed_shots_api["personId"].isin(rated_players)
]


In [31]:
made_shots_api.groupby(
    ["personId", "shot_family", "shot_subtype"]
).size()


personId  shot_family  shot_subtype
203999    2PT          floater         2
                       hook            1
                       jump_shot       1
                       layup           1
          3PT          jump_shot       1
1628392   2PT          layup           2
1631212   2PT          dunk            1
                       floater         2
                       layup           1
                       pullup          2
          3PT          jump_shot       5
dtype: int64

In [32]:
import pandas as pd

rows = []

for game_id, players in app_data.items():
    for person_id, pdata in players.items():
        for ev in pdata["events"]:
            ui = ev["ui_event"].lower()

            # keep ONLY made shots
            if any(x in ui for x in ["missed", "turnover", "steal", "free throw", "block"]):
                continue

            rows.append({
                "personId": person_id,
                "period": ev["period"],
                "clock": ev["clock"],
                "delta": ev["delta"]
            })

shots_made_app = pd.DataFrame(rows)


In [33]:
print(type(shots_made_app))
shots_made_app.head()


<class 'pandas.core.frame.DataFrame'>


,personId,period,clock,delta
0,1631212,1,10:51,0.30
1,1631212,1,6:18,0.23
2,1631212,1,2:31,0.19
3,1631212,2,3:29,0.12
4,1631212,2,1:42,0.24


In [34]:
shots_made_app["personId"].value_counts()


personId
1631212    17
203999     14
1628392     3
Name: count, dtype: int64

In [35]:
made_shots_api


,personId,period,clock,actionType,subType,shotDistance,shotValue,description,shot_family,shot_subtype
15,1628392,1,PT10M12.00S,Made Shot,Tip Layup Shot,0,2,Hartenstein Tip Layup Shot (2 PTS),2PT,layup
20,1631212,1,PT08M52.00S,Made Shot,Jump Shot,25,3,Watson 25' 3PT Jump Shot (3 PTS) (Pickett 1 AST),3PT,jump_shot
27,1631212,1,PT07M39.00S,Made Shot,Cutting Dunk Shot,0,2,Watson Cutting Dunk Shot (5 PTS) (Jokic 2 AST),2PT,dunk
51,203999,1,PT05M10.00S,Made Shot,Turnaround Hook Shot,7,2,Jokic 7' Turnaround Hook Shot (2 PTS),2PT,hook
66,1631212,1,PT03M46.00S,Made Shot,Pullup Jump shot,8,2,Watson 8' Pullup Jump Shot (7 PTS) (Murray 2 AST),2PT,pullup
79,1631212,1,PT03M07.00S,Made Shot,Jump Shot,24,3,Watson 24' 3PT Jump Shot (10 PTS) (Valanciunas...,3PT,jump_shot
214,1631212,2,PT02M24.00S,Made Shot,Driving Floating Jump Shot,3,2,Watson 3' Driving Floating Jump Shot (12 PTS),2PT,floater
247,203999,2,PT00M21.70S,Made Shot,Floating Jump shot,6,2,Jokic 6' Floating Jump Shot (6 PTS) (Murray 3 ...,2PT,floater
253,203999,3,PT11M48.00S,Made Shot,Jump Shot,19,2,Jokic 19' Jump Shot (8 PTS) (Murray 4 AST),2PT,jump_shot
269,1628392,3,PT09M20.00S,Made Shot,Cutting Layup Shot,3,2,Hartenstein 3' Cutting Layup Shot (4 PTS) (Gil...,2PT,layup


In [36]:

def normalize_clock_api(clock):
    """
    Convert NBA API clock format:
    PT11M38.00S -> 11:38
    """
    if clock is None:
        return None

    clock = clock.replace("PT", "").replace("S", "")
    mins, secs = clock.split("M")
    return f"{int(mins)}:{int(float(secs)):02d}"


In [37]:
made_shots_api["clock_norm"] = made_shots_api["clock"].apply(normalize_clock_api)


In [38]:
shots_made_app["clock_norm"] = shots_made_app["clock"]


In [39]:
shots_with_delta = pd.merge(
    made_shots_api,
    shots_made_app[["personId", "period", "clock_norm", "delta"]],
    on=["personId", "period", "clock_norm"],
    how="inner"
)


In [40]:
shots_with_delta[
    [
        "personId",
        "period",
        "clock_norm",
        "shotValue",
        "shot_subtype",
        "shotDistance",
        "delta"
    ]
]


,personId,period,clock_norm,shotValue,shot_subtype,shotDistance,delta
0,1628392,1,10:12,2,layup,0,0.20
1,203999,1,5:10,2,hook,7,0.19
2,203999,3,11:48,2,jump_shot,19,0.32
3,1628392,3,9:20,2,layup,3,0.20
4,203999,3,7:20,2,layup,0,0.22
5,203999,3,6:02,3,jump_shot,26,0.41
6,203999,4,9:36,2,floater,9,0.21


In [41]:
with open ('shots_made.pkl', 'wb') as f:
    pickle.dump(app_data, f)

In [42]:
shots_missed_df.sort_values(by="delta", ascending=False)


,player,period,clock,ui_event,delta
4,Watson,3,2:52,Missed 12' drive game-tying floater,0.15
10,Hartenstein,4,7:24,Missed 27' three,0.15
11,Jokic,1,6:21,Missed 24' corner three,0.15
7,Hartenstein,1,10:14,Missed 15' fadeaway,0.15
0,Watson,1,8:54,Missed 15' drive floater,0.07
3,Watson,3,8:11,Missed 27' lead-taking three,0.07
2,Watson,2,31.1,Missed 30' bomb,0.07
1,Watson,1,5:59,Missed 24' lead-taking three,0.07
5,Watson,3,1:57,Missed 24' lead-taking three,0.07
8,Hartenstein,2,1:20,Missed drive reverse layup,0.07


In [43]:
shots_made_app[shots_made_app["personId"] == 1631212]


,personId,period,clock,delta,clock_norm
0,1631212,1,10:51,0.30,10:51
1,1631212,1,6:18,0.23,6:18
2,1631212,1,2:31,0.19,2:31
3,1631212,2,3:29,0.12,3:29
4,1631212,2,1:42,0.24,1:42
5,1631212,2,26.3,0.13,26.3
6,1631212,3,11:30,0.17,11:30
7,1631212,3,11:00,0.22,11:00
8,1631212,3,6:56,0.27,6:56
9,1631212,3,2:37,0.16,2:37


In [44]:
made_shots_api[made_shots_api["personId"] == 1631212][
    ["period", "clock", "clock_norm", "shotValue", "shot_subtype"]
]


,period,clock,clock_norm,shotValue,shot_subtype
20,1,PT08M52.00S,8:52,3,jump_shot
27,1,PT07M39.00S,7:39,2,dunk
66,1,PT03M46.00S,3:46,2,pullup
79,1,PT03M07.00S,3:07,3,jump_shot
214,2,PT02M24.00S,2:24,2,floater
277,3,PT08M33.00S,8:33,3,jump_shot
279,3,PT07M59.00S,7:59,2,pullup
356,3,PT00M27.70S,0:27,2,floater
388,4,PT08M39.00S,8:39,3,jump_shot
426,4,PT05M26.00S,5:26,2,layup


In [52]:
rows = []

for game_id, game_data in app_data.items():
    for person_id, pdata in game_data.items():
        for ev in pdata["events"]:
            rows.append({
                "game_id": game_id,
                "personId": person_id,
                "period": ev["period"],
                "clock": ev["clock"],
                "ui_event": ev["ui_event"],
                "delta": ev["delta"]
            })


In [54]:
import pandas as pd

df = pd.DataFrame(rows)
df.head()


,game_id,personId,period,clock,ui_event,delta
0,0022500711,1631212,1,10:51,24' corner three,0.30
1,0022500711,1631212,1,8:54,Missed 15' drive floater,0.07
2,0022500711,1631212,1,6:18,25' fastbreak lead-taking three,0.23
3,0022500711,1631212,1,5:59,Missed 24' lead-taking three,0.07
4,0022500711,1631212,1,4:43,Turnover - Lost ball,-0.18


In [55]:
shot_df = df[
    df["ui_event"].str.contains("three|dunk|layup|jumper|shot", case=False, na=False)
]


In [56]:
shot_df[["ui_event", "delta"]].sample(20)


,ui_event,delta
46,24' corner three,0.28
32,Missed drive reverse layup,0.07
25,26' three,0.25
19,Missed 24' lead-taking three,0.07
23,24' corner three,0.29
2,25' fastbreak lead-taking three,0.23
40,Dunk,0.14
0,24' corner three,0.30
9,24' three,0.24
18,Fastbreak game-tying dunk,0.16


In [57]:
shot_df[
    ~shot_df["ui_event"].str.contains("three|dunk|layup|jumper", case=False, na=False)
][["ui_event", "delta"]].head(20)


,ui_event,delta


In [58]:
SHOT_KEYWORDS = [
    "three", "dunk", "layup", "jumper", "floater",
    "hook", "tip", "putback", "alley-oop"
]

def is_shot(text):
    if not isinstance(text, str):
        return False
    t = text.lower()
    return any(k in t for k in SHOT_KEYWORDS)


In [59]:
df["is_shot"] = df["ui_event"].apply(is_shot)
df[df["is_shot"]][["ui_event", "delta"]].sample(20)


,ui_event,delta
1,Missed 15' drive floater,0.07
30,Putback layup,0.20
77,Missed 27' three,0.07
57,Putback layup,0.22
32,Missed drive reverse layup,0.07
45,27' three,0.18
66,Missed 24' corner three,0.07
37,Missed 27' three,0.15
61,10' drive floater,0.21
20,Alley-oop fastbreak layup,0.13


In [60]:
import re

def extract_distance(text):
    if not isinstance(text, str):
        return None
    m = re.search(r"(\d{1,2})'", text)
    return int(m.group(1)) if m else None


In [62]:
shot_df.groupby("personId")["delta"].mean().sort_values()


personId
203501     0.103333
1628392    0.157143
203999     0.170714
1631212    0.196000
Name: delta, dtype: float64

In [64]:
from collections import Counter

tokens = Counter()

for text in shot_df["ui_event"].dropna():
    for w in re.findall(r"[a-zA-Z\-]+", text.lower()):
        tokens[w] += 1


In [65]:
tokens.most_common(40)


[('three', 25),
 ('missed', 14),
 ('layup', 11),
 ('corner', 7),
 ('fastbreak', 7),
 ('lead-taking', 6),
 ('drive', 4),
 ('dunk', 3),
 ('putback', 2),
 ('and-', 1),
 ('game-tying', 1),
 ('alley-oop', 1),
 ('reverse', 1),
 ('blocked', 1)]

In [67]:
# STEP 1: Identify made vs missed shots

def is_missed(ui_event):
    return isinstance(ui_event, str) and ui_event.lower().startswith("missed")

shot_df["is_missed"] = shot_df["ui_event"].apply(is_missed)
shot_df["is_made"] = ~shot_df["is_missed"]

# Sanity check
shot_df.groupby("is_made")["delta"].describe()


C:\Users\ishan\AppData\Local\Temp\ipykernel_6260\1241119991.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df["is_missed"] = shot_df["ui_event"].apply(is_missed)
C:\Users\ishan\AppData\Local\Temp\ipykernel_6260\1241119991.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df["is_made"] = ~shot_df["is_missed"]


,count,mean,std,min,25%,50%,75%,max
is_made,,,,,,,,
False,14.0,0.081429,0.029051,0.07,0.07,0.07,0.07,0.15
True,25.0,0.224000,0.067454,0.13,0.17,0.22,0.26,0.41


In [68]:
# STEP 1: Identify made vs missed shots

def is_missed(ui_event):
    return isinstance(ui_event, str) and ui_event.lower().startswith("missed")

shot_df["is_missed"] = shot_df["ui_event"].apply(is_missed)
shot_df["is_made"] = ~shot_df["is_missed"]

# Sanity check
shot_df.groupby("is_made")["delta"].describe()


C:\Users\ishan\AppData\Local\Temp\ipykernel_6260\1241119991.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df["is_missed"] = shot_df["ui_event"].apply(is_missed)
C:\Users\ishan\AppData\Local\Temp\ipykernel_6260\1241119991.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df["is_made"] = ~shot_df["is_missed"]


,count,mean,std,min,25%,50%,75%,max
is_made,,,,,,,,
False,14.0,0.081429,0.029051,0.07,0.07,0.07,0.07,0.15
True,25.0,0.224000,0.067454,0.13,0.17,0.22,0.26,0.41


In [69]:
# STEP 2: Freeze attempt value and compute residual (learnable target)

def attempt_value(row):
    # Missed shots with higher credit (difficulty / leverage)
    if row["is_missed"] and row["delta"] > 0.07:
        return 0.15
    # All other shot attempts
    return 0.07

shot_df["attempt_value"] = shot_df.apply(attempt_value, axis=1)

# Residual = value of making the shot (what we will learn)
shot_df["residual"] = shot_df["delta"] - shot_df["attempt_value"]

# Sanity check: residuals for missed shots should be ~0
shot_df.groupby("is_made")["residual"].describe()


C:\Users\ishan\AppData\Local\Temp\ipykernel_6260\769052161.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df["attempt_value"] = shot_df.apply(attempt_value, axis=1)
C:\Users\ishan\AppData\Local\Temp\ipykernel_6260\769052161.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df["residual"] = shot_df["delta"] - shot_df["attempt_value"]


,count,mean,std,min,25%,50%,75%,max
is_made,,,,,,,,
False,14.0,0.000,0.000000,0.00,0.0,0.00,0.00,0.00
True,25.0,0.154,0.067454,0.06,0.1,0.15,0.19,0.34


In [70]:
# STEP 3: Build shot feature columns (no learning yet)

# Shot value
shot_df["is_three"] = shot_df["ui_event"].str.contains("three", case=False, na=False)
shot_df["is_two"] = ~shot_df["is_three"]

# Shot types (binary flags)
shot_df["is_dunk"] = shot_df["ui_event"].str.contains("dunk", case=False, na=False)
shot_df["is_layup"] = shot_df["ui_event"].str.contains("layup", case=False, na=False)
shot_df["is_putback"] = shot_df["ui_event"].str.contains("putback", case=False, na=False)
shot_df["is_floater"] = shot_df["ui_event"].str.contains("floater", case=False, na=False)
shot_df["is_hook"] = shot_df["ui_event"].str.contains("hook", case=False, na=False)
shot_df["is_jump"] = shot_df["ui_event"].str.contains("jump", case=False, na=False)

# Distance (numeric, untouched)
shot_df["distance"] = shot_df["ui_event"].apply(extract_distance)

# Look at made shots only
shot_df[shot_df["is_made"]][[
    "ui_event",
    "residual",
    "is_three",
    "is_dunk",
    "is_layup",
    "is_putback",
    "is_floater",
    "is_hook",
    "is_jump",
    "distance"
]].head(10)


C:\Users\ishan\AppData\Local\Temp\ipykernel_6260\1336482957.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df["is_three"] = shot_df["ui_event"].str.contains("three", case=False, na=False)
C:\Users\ishan\AppData\Local\Temp\ipykernel_6260\1336482957.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df["is_two"] = ~shot_df["is_three"]
C:\Users\ishan\AppData\Local\Temp\ipykernel_6260\1336482957.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

,ui_event,residual,is_three,is_dunk,is_layup,is_putback,is_floater,is_hook,is_jump,distance
0,24' corner three,0.23,True,False,False,False,False,False,False,24.0
2,25' fastbreak lead-taking three,0.16,True,False,False,False,False,False,False,25.0
9,24' three,0.17,True,False,False,False,False,False,False,24.0
11,Fastbreak layup,0.06,False,False,True,False,False,False,False,NaN
13,Drive lead-taking layup and-1,0.15,False,False,True,False,False,False,False,NaN
15,26' three,0.20,True,False,False,False,False,False,False,26.0
18,Fastbreak game-tying dunk,0.09,False,True,False,False,False,False,False,NaN
20,Alley-oop fastbreak layup,0.06,False,False,True,False,False,False,False,NaN
21,24' corner three,0.19,True,False,False,False,False,False,False,24.0
22,27' three,0.18,True,False,False,False,False,False,False,27.0


In [71]:
# STEP 4: Define 3PT distance buckets (made shots only)

# Inspect distribution (made 3PTs only)
made_3pt = shot_df[(shot_df["is_made"]) & (shot_df["is_three"])].copy()

made_3pt["distance"].describe(), made_3pt["distance"].value_counts().sort_index()


(count    15.000000
 mean     25.400000
 std       1.183216
 min      24.000000
 25%      24.000000
 50%      26.000000
 75%      26.000000
 max      27.000000
 Name: distance, dtype: float64,
 distance
 24.0    5
 25.0    2
 26.0    5
 27.0    3
 Name: count, dtype: int64)

In [72]:
# Create 3PT distance buckets
def three_pt_bucket(d):
    if pd.isna(d):
        return None
    if d <= 24:
        return "short_3"
    elif d <= 26:
        return "standard_3"
    else:
        return "deep_3"

shot_df["three_pt_bucket"] = shot_df["distance"].apply(three_pt_bucket)

# Quick check
shot_df[shot_df["is_three"] & shot_df["is_made"]][
    ["ui_event", "distance", "three_pt_bucket", "residual"]
].head(10)


C:\Users\ishan\AppData\Local\Temp\ipykernel_6260\210299830.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df["three_pt_bucket"] = shot_df["distance"].apply(three_pt_bucket)


,ui_event,distance,three_pt_bucket,residual
0,24' corner three,24.0,short_3,0.23
2,25' fastbreak lead-taking three,25.0,standard_3,0.16
9,24' three,24.0,short_3,0.17
15,26' three,26.0,standard_3,0.20
21,24' corner three,24.0,short_3,0.19
22,27' three,27.0,deep_3,0.18
23,24' corner three,24.0,short_3,0.22
25,26' three,26.0,standard_3,0.18
33,26' three,26.0,standard_3,0.27
44,25' corner three,25.0,standard_3,0.11


In [73]:
# STEP 5: Build regression-ready dataset (made shots only)

# Use made shots only
reg_df = shot_df[shot_df["is_made"]].copy()

# Target: pure make value
y = reg_df["residual"]

# Feature columns (explicit and interpretable)
X = pd.DataFrame({
    # Shot value
    "is_three": reg_df["is_three"].astype(int),

    # Shot type flags (2PT mechanics)
    "is_dunk": reg_df["is_dunk"].astype(int),
    "is_layup": reg_df["is_layup"].astype(int),
    "is_putback": reg_df["is_putback"].astype(int),
    "is_floater": reg_df["is_floater"].astype(int),
    "is_hook": reg_df["is_hook"].astype(int),
    "is_jump": reg_df["is_jump"].astype(int),

    # 3PT distance buckets (one-hot, reference = standard_3)
    "short_3": (reg_df["three_pt_bucket"] == "short_3").astype(int),
    "deep_3": (reg_df["three_pt_bucket"] == "deep_3").astype(int),
})

# Inspect
X.head(), y.describe()


(    is_three  is_dunk  is_layup  is_putback  is_floater  is_hook  is_jump  \
 0          1        0         0           0           0        0        0   
 2          1        0         0           0           0        0        0   
 9          1        0         0           0           0        0        0   
 11         0        0         1           0           0        0        0   
 13         0        0         1           0           0        0        0   
 
     short_3  deep_3  
 0         1       0  
 2         0       0  
 9         1       0  
 11        0       0  
 13        0       0  ,
 count    25.000000
 mean      0.154000
 std       0.067454
 min       0.060000
 25%       0.100000
 50%       0.150000
 75%       0.190000
 max       0.340000
 Name: residual, dtype: float64)

In [74]:
# STEP 6: Constrained linear regression for shot make value

import numpy as np
from scipy.optimize import lsq_linear

# Design matrix
X_mat = X.values
y_vec = y.values

feature_names = X.columns.tolist()

# ---- CONSTRAINTS ----
# Lower and upper bounds for each coefficient (same order as X columns)

bounds = {
    "is_three":   (0.05, 0.30),   # 3PT must add value
    "is_dunk":    (0.05, 0.25),
    "is_layup":   (0.03, 0.20),
    "is_putback": (0.04, 0.22),
    "is_floater": (0.03, 0.20),
    "is_hook":    (0.03, 0.20),
    "is_jump":    (0.02, 0.18),
    "short_3":    (0.00, 0.08),   # corner bonus
    "deep_3":     (-0.05, 0.02),  # deep shots slightly penalized
}

lb = np.array([bounds[f][0] for f in feature_names])
ub = np.array([bounds[f][1] for f in feature_names])

# ---- FIT ----
res = lsq_linear(X_mat, y_vec, bounds=(lb, ub))

# ---- RESULTS ----
coef_df = pd.DataFrame({
    "feature": feature_names,
    "coefficient": res.x
}).sort_values("coefficient", ascending=False)

coef_df


,feature,coefficient
0,is_three,0.194000
4,is_floater,0.130000
5,is_hook,0.130000
6,is_jump,0.120000
2,is_layup,0.099998
1,is_dunk,0.086667
3,is_putback,0.040008
7,short_3,0.010000
8,deep_3,-0.050000


In [76]:
# STEP 7: Compute base predicted shot delta (no context)

# ---- BASE COEFFICIENTS (LOCKED) ----
BASE_COEF = {
    "is_three": 0.194,
    "is_floater": 0.130,
    "is_hook": 0.130,
    "is_jump": 0.120,
    "is_layup": 0.100,
    "is_dunk": 0.087,
    "is_putback": 0.040,
    "short_3": 0.010,
    "deep_3": -0.050,
}

ATTEMPT_BASE = 0.07
ATTEMPT_HARD = 0.15

def predicted_base_delta(row):
    # attempt value
    attempt = ATTEMPT_HARD if (row["is_missed"] and row["delta"] > 0.07) else ATTEMPT_BASE

    # make bonus
    make_bonus = 0.0
    for f, w in BASE_COEF.items():
        if f in row and row[f]:
            make_bonus += w

    return attempt + make_bonus

shot_df["predicted_base_delta"] = shot_df.apply(predicted_base_delta, axis=1)

# Inspect a few made shots
shot_df[shot_df["is_made"]][
    ["ui_event", "delta", "predicted_base_delta"]
].head(10)


C:\Users\ishan\AppData\Local\Temp\ipykernel_6260\1407361038.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df["predicted_base_delta"] = shot_df.apply(predicted_base_delta, axis=1)


,ui_event,delta,predicted_base_delta
0,24' corner three,0.30,0.264
2,25' fastbreak lead-taking three,0.23,0.264
9,24' three,0.24,0.264
11,Fastbreak layup,0.13,0.170
13,Drive lead-taking layup and-1,0.22,0.170
15,26' three,0.27,0.264
18,Fastbreak game-tying dunk,0.16,0.157
20,Alley-oop fastbreak layup,0.13,0.170
21,24' corner three,0.26,0.264
22,27' three,0.25,0.264


In [77]:
# STEP 8: Compute raw context uplift

shot_df["context_uplift"] = shot_df["delta"] - shot_df["predicted_base_delta"]

# Look only at made shots with context words
context_view = shot_df[
    (shot_df["is_made"]) &
    (shot_df["ui_event"].str.contains("fastbreak|lead-taking|game-tying|and-1", 
                                      case=False, na=False))
][["ui_event", "delta", "predicted_base_delta", "context_uplift"]]

context_view


C:\Users\ishan\AppData\Local\Temp\ipykernel_6260\2606040120.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df["context_uplift"] = shot_df["delta"] - shot_df["predicted_base_delta"]


,ui_event,delta,predicted_base_delta,context_uplift
2,25' fastbreak lead-taking three,0.23,0.264,-0.034
11,Fastbreak layup,0.13,0.170,-0.040
13,Drive lead-taking layup and-1,0.22,0.170,0.050
18,Fastbreak game-tying dunk,0.16,0.157,0.003
20,Alley-oop fastbreak layup,0.13,0.170,-0.040
63,Fastbreak layup,0.17,0.170,0.000


In [79]:
from nba_api.stats.endpoints import playbyplayv3
import pandas as pd

game_id = "0022500711"

pbp = playbyplayv3.PlayByPlayV3(game_id=game_id)
pbp_df = pbp.get_data_frames()[0]

pbp_df.head()


,gameId,actionNumber,clock,period,teamId,teamTricode,personId,playerName,playerNameI,xLegacy,...,scoreHome,scoreAway,pointsTotal,location,description,actionType,subType,videoAvailable,shotValue,actionId
0,0022500711,2,PT12M00.00S,1,0,,0,,,0,...,0,0,0,,Start of 1st Period (9:43 PM EST),period,start,1,0,1
1,0022500711,4,PT12M00.00S,1,1610612743,DEN,203999,Jokić,N. Jokić,0,...,,,0,h,Jump Ball Jokic vs. Holmgren: Tip to Wallace,Jump Ball,,1,0,2
2,0022500711,7,PT11M46.00S,1,1610612760,OKC,1628392,Hartenstein,I. Hartenstein,0,...,,,0,v,Hartenstein Bad Pass Turnover (P1.T1),Turnover,Bad Pass,1,0,3
3,0022500711,7,PT11M46.00S,1,1610612743,DEN,1631212,Watson,P. Watson,0,...,,,0,h,Watson STEAL (1 STL),,,1,0,4
4,0022500711,9,PT11M38.00S,1,1610612743,DEN,1631212,Watson,P. Watson,13,...,,,0,h,MISS Watson 3' Cutting Layup Shot,Missed Shot,Cutting Layup Shot,1,2,5


In [84]:
pbp_df[["scoreHome", "scoreAway"]] = (
    pbp_df[["scoreHome", "scoreAway"]]
    .replace("", pd.NA)
    .astype("float")
    .ffill()
    .fillna(0)
)

pbp_df["score_for"] = pbp_df.apply(
    lambda r: r["scoreHome"] if r["location"] == "h" else r["scoreAway"],
    axis=1
)

pbp_df["score_against"] = pbp_df.apply(
    lambda r: r["scoreAway"] if r["location"] == "h" else r["scoreHome"],
    axis=1
)

pbp_df[["teamTricode", "location", "scoreHome", "scoreAway", "score_for", "score_against"]].head(12)


TypeError: float() argument must be a string or a real number, not 'NAType'